# Biostat 257 Homework 3

**Due May 6 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, haswell)


In [2]:
#import Pkg; Pkg.add("Distributions")

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

#### Solution

$$E[\mathbf{Y}_i] = \mathbf{X}_i \boldsymbol{\beta};$$ $$var(\mathbf{Y}_i) = \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 \mathbf{I}_{n_i} = \mathbf{V}_i$$

Therefore, $\mathbf{Y}_i \sim N_{n_i}(\mathbf{X}_i \boldsymbol{\beta}, \mathbf{V}_i)$ by convolution of normals

$L(\mathbf{Y}_i, \mathbf{X}_i, \mathbf{Z}_i| \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \frac{1}{(2 \pi)^{n_i/2} (\det{\mathbf{V}_i})^{1/2}} \exp({-\frac{1}{2}(\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})})$

$l(\mathbf{Y}_i, \mathbf{X}_i, \mathbf{Z}_i| \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = -\frac{n_i}{2} \log (2 \pi) -\frac{1}{2} \log(\det{\mathbf{V}_i}) -\frac{1}{2} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [28]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    s_q2       :: Vector{T}
    s_p2       :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    s_qq2      :: Matrix{T}
    s_qq3      :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    s_q2       = similar(storage_q)
    s_p2       = similar(storage_p)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    s_qq2      = similar(ztz)
    s_qq3      = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_q, s_q2, s_p2, xtx, ztx, ztz, storage_qq, s_qq2, s_qq3)
end


LoadError: invalid redefinition of constant LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

**Step 1: Prototype**

In [3]:
function logl_proto!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood
    c = - (n / 2) * log(2π * σ²) # constant, should not allocate
    sum1 = det(I + (L * L') * obs.ztz / σ²) # determinant
    sum2 = transpose(obs.y) * (obs.y) - 2transpose(obs.y) * obs.X * β + 
        transpose(β) * obs.xtx * β
    star = inv(inv(L * L') + obs.ztz / σ²)
    sum3 = transpose(obs.y) * obs.Z * star * transpose(obs.Z) * obs.y - 
        2transpose(obs.y) * obs.Z * star * obs.ztx * β + 
        transpose(β) * transpose(obs.ztx) * star * obs.ztx * β
    loglik = c - 0.5 * log(sum1) - 0.5 * sum2 / σ² + 0.5 * sum3 / (σ²)^2
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return loglik
end

logl_proto! (generic function with 1 method)

**Step 2: Improve allocation, benchmarking, etc.**

In [24]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    
    # constant term
    c = - (n / 2) * log(2π * σ²) # constant, should not allocate
    
    mul!(obs.storage_qq, transpose(L), obs.ztz / σ²) # stores L'Z'Z
    mul!(obs.s_qq2, obs.storage_qq, L) # stores L'Z'ZL

    # adding identity matrix to L'Z'ZL/σ²
    for i = 1:q, j = 1:q
        @inbounds if i == j
            obs.s_qq2[i, j] += 1
        end
    end
    
    # decompose the lower-dimensional inverse term
    chol = cholesky!(Symmetric(obs.s_qq2)) 
    
    # error term
    mul!(obs.storage_p, transpose(obs.X), obs.y) # stores X'y (px1)
    mul!(obs.s_p2, obs.xtx, β) # stores X'Xβ
    error = dot(obs.y, obs.y) - 2dot(obs.storage_p, β) + dot(β, obs.s_p2)
    
    mul!(obs.storage_q, transpose(obs.Z), obs.y) # stores Z'y (qx1)
    mul!(obs.s_q2, obs.ztx, β) # stores Z'Xβ
    axpy!(-1, obs.s_q2, obs.storage_q) # Z'Y - Z'Xβ stored in storage_q 
    
    return c - 0.5logdet(chol) - 0.5error / σ² + 
        0.5dot(L' * obs.storage_q, chol \ L' * obs.storage_q) / (σ²)^2
    
    #return - (n / 2) * log(2π * σ²) - 0.5logdet(chol) - 
    #    (1 / 2σ²) * (dot(obs.y, obs.y) - 2dot(obs.storage_p, β) + dot(β, obs.s_p2)) +
    #    0.5dot(L' * obs.storage_q, chol \ L' * obs.storage_q) / (σ²)^2
    
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    #return loglik
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [4]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [5.0e-324, 5.0e-324, 5.0e-324, 5.0e-324, 0.0], [2.2662150154e-314, 2.3277738084e-314, 5.0e-324], [2.2569630394e-314, 2.256976154e-314, 2.

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [5]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [25]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058276

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [7]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [8]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 2724 samples with 1 evaluation.
 Range (min … max):  1.481 ms …  19.208 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.688 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.808 ms ± 736.435 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▃▃██▆▅▅▃▄▃▃▂▂▂▁▁                                         ▁
  ▃▇████████████████████▆▇▇▆▇▅▆▆▅▃▇▃▃▃▅▃▄▃▅▅▃▁▄▄▄▅▄▃▄▃▃▄▁▁▃▁▃ █
  1.48 ms      Histogram: log(frequency) by time      3.27 ms <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [26]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.907 μs … 108.376 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.079 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.492 μs ±   2.532 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▅▂▁                     ▁                                 ▂
  ███████▇▆▁▄▄▇▆▆▄▇▃▄▄▄▆▅▆▇███▇▇▇▆▆▆▆▅▆▄▆▅▆▄▅▅▄▄▃▁▄▅▄▅▄▁▃▃▁▄▃ █
  6.91 μs      Histogram: log(frequency) by time      15.7 μs <

 Memory estimate: 464 bytes, allocs estimate: 6.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [27]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

7.155417431840656

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [23]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.546875

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.